# Datengenerator und Bewertung
    Szenario der Bewährungsauflagen Bewertung mit 5 Attributen(Name, Hautfarbe, Laufende_Strafe in Jahre, Geschlecht und Härte des Vergehens)
    Dabei können Hautfarbe und Geschlecht durch Bias beeinflusst werden.

In [2]:
import numpy as np
from faker import Faker
import pandas as pd
from datetime import datetime
import random

fake = Faker()

#### Eine Funktion zum erstellen der synthetischen Daten.
    
    Die Parameter num und seed geben dabei zum einen die Anzahl an Datensätzen an und zum anderen den seed der "künstlichen" Wahrscheinlichkeit.
    Es wird pro Datensatz ein geschlecht(M,W) daraufbasierend die haerte(Leicht, Mittel, Hart), die hautfarbe(Weiß, Schwarz) und einen Namen zufällig ausgewählt.
    Zu guter Letzt wird noch die Laufende_Strafe(1,2,3,4,5) ohne Fremdeinwirkung zufällig ausgewählt.

In [3]:
def create_fake_data(num = 10, seed = 123):
    #Setting the seed for the probability functions
    np.random.seed(seed)
    fake.seed_instance(seed)
    #Defining the Output Array
    output = []
    #Loop over the number of requests to be created
    for x in range(num):
        #Random choice of sex with specified probability
        sex = np.random.choice(["M", "W"], p=[0.927, 0.073])
        #Random choice of hardness taking into account gender
        hardnes = np.random.choice(["Leicht", "Mittel", "Hart"], p=[0.266, 0.169, 0.565]) if sex=="M" else np.random.choice(["Leicht", "Mittel", "Hart"], p=[0.361, 0.264, 0.375])
        #Random choice of skin colour taking into account gender
        skin = np.random.choice(["Weiß", "Schwarz"], p=[0.459, 0.541]) if sex=="M" else np.random.choice(["Weiß", "Schwarz"], p=[0.715, 0.285])
        #Add entry to the array
        output.append(
            {
                #Name of the person based on gender
                "Name": fake.first_name_male() if sex=="M" else fake.first_name_female(),
                "Hautfarbe": skin,
                #Random skin colour
                "Laufende_Strafe": np.random.choice([1,2,3,4,5]),
                "Geschlecht": sex,
                "Haerte_des_Vergehens": hardnes
            }  
        )
    #Return of the output array with one request per entry
    return output

#### Funktion zum Erstellen der Regeln nach welchen ein Bewerter bewerten soll.
    Parameter: antrag_werte ist ein Array an den zur Bewertung relevanten Werte wie z.B. die Härte des Vergehens
    antrag_bias ist ein Array an den mit Bias sinnvoll Beinflussbaren Werte wie z.B. Geschlecht
    bias ist der letztendliche Bias welcher ausgeübt werden soll z.B. Hautfarbe:Schwarz bedeutet Diskriminierung gegenüber Schwarzen.
    Es werden 4 Regeln als Dictionarys erstellt:
    regel_pos: Sind die Werte für eine positive Bewertung eines Antrags ohne Bias
    regel_neg: Sind die Werte für eine negative Bewertung eines Antrags ohne Bias
    regel_pos_bias: Sind die Werte für eine positive Bewertung eines Antrags mit Betrachtung des Bias
    regel_neg_bias: Sind die Werte für eine negative Bewertung eines Antrags mit Betrachtung des Bias
    Diese Regeln werden am Ende zurückgegeben

In [4]:
def create_Rules(request_values, request_bias, bias):
    #Creating the Rule Dictionarys
    rule_pos = {}
    rule_neg = {}
    rule_neg_bias = {}
    rule_pos_bias = {}
    #For each possible request value categoriser (Härte des Vergehens, Laufende Strafe)
    for key in request_values.keys():
        #The following characteristics exist(härte: Leicht,Mittel,Hart)
        values = request_values[key]
        #Determine mean value of the expressions to distribute 50/50 between positive and negative rules
        length = int(len(values)/2)+1
        pos_values = []
        #Adding the positive first values
        for x in range(length):
            pos_values.append(values[x])
        #Add the expressions with the category to the regel_pos Dict
        rule_pos[key] = pos_values
        #Add the remaining expressions of the category to the rule_neg
        neg_values = []
        for y in range(len(values)):
            if(y >= length):
                neg_values.append(values[y])
        rule_neg[key] = neg_values
    #The distribution of the rules remains the same regardless of whether bias or not, so copy deep for now.
    rule_pos_bias = rule_pos.copy()
    rule_neg_bias = rule_neg.copy()
    #If there is to be no bias then skip
    if(bias != None):
        #For each possible bias category
        for key in request_bias.keys():
            #If the value is equal to the specified bias parameter
            if(bias.keys().__contains__(key)):
                #Adding the negative specified bias rule
                rule_neg_bias[key] = bias[key]
                neg_bias = bias[key]
                rule_pos_bias[key] = []
                #Adding the remaining opposite positive bias rule
                for val in request_bias[key]:
                    if not(neg_bias.__contains__(val)):
                        rule_pos_bias[key].append(val)
    #Return of all 4 generated Rule Dicts
    return(rule_pos, rule_neg, rule_pos_bias, rule_neg_bias)

#### Testzelle zum prüfen der erzeugten Regeln

In [5]:
#Defining the dictionaries for the bias, the request values and the request bias values
bias = {
    "Hautfarbe": ["Schwarz"]
}
request_values = {
    "Laufende_Strafe": [1,2,3,4,5],
    "Haerte_des_Vergehens": ["Leicht", "Mittel", "Hart"]
}
request_bias = {
    "Hautfarbe": ["Schwarz", "Weiß"],
    "Geschlecht": ["M", "W"]
}
#Create and output sample rules
r_pos, r_neg, rb_pos, rb_neg = create_Rules(request_values, request_bias, bias)
print(r_pos)
print(r_neg)
print(rb_pos)
print(rb_neg)

{'Laufende_Strafe': [1, 2, 3], 'Haerte_des_Vergehens': ['Leicht', 'Mittel']}
{'Laufende_Strafe': [4, 5], 'Haerte_des_Vergehens': ['Hart']}
{'Laufende_Strafe': [1, 2, 3], 'Haerte_des_Vergehens': ['Leicht', 'Mittel'], 'Hautfarbe': ['Weiß']}
{'Laufende_Strafe': [4, 5], 'Haerte_des_Vergehens': ['Hart'], 'Hautfarbe': ['Schwarz']}


#### Generieren eines ziemlich zufälligen Seeds
    Generieren eines Seeds beruhend auf den Angaben der Zeit.
    So ist bei jeder generierung der Daten ein anderer Seed verfügbar und es entstehen "wirkliche" Zufallswerte

In [6]:
def generate_seed():
    now = datetime.now()
    #Calculate seed number from a few time data
    seed = (now.day * now.minute * now.second * now.month * now.year * now.hour) / now.microsecond 
    #If a negative or 0 value is the result, a "simpler" replacement seed is generated. 
    if(seed <= 0):
        seed = now.day * (now.minute + 1)
    return seed

#### Klasse für das erstellen eines Bewerters 
    Beinhaltet Methoden zum einen zum erstellen des Bewerter Objekts und zum anderen die Methode bewerte, um einen Antrag nach den eigenen Regeln zu Bewerten. So hat jeder Bewerte seine eigenen Regln und einen Bias oder nicht.
    

In [7]:
class Evaluator:
    #Creating a evaluator with its own rules and a bias or not
    def __init__(self, rule_pos, rule_neg, bias, percentage=0.2):
        self.rule_pos = rule_pos
        self.rule_neg = rule_neg
        self.bias = bias
        self.bias_percentage = percentage
    #Function to evaluate a submitted application with or without bias
    def rate(self, request, bias):
        #First 50/50 distribution
        pos = 50
        #Calculate the proportion according to which the decision is influenced positively or negatively.
        prop = 45/self.rule_pos.__len__()
        #Depending on how the rules match the request, the weight of the positive evaluation is shifted.
        for key in self.rule_pos.keys():
            if(self.rule_pos[key].__contains__(request[key])):
                pos += prop
            else:
                pos -= prop
        try:
            #If a bias is present, this is additionally taken into account with the Parameter in %
            if(self.bias):
                for b in bias:
                    if(bias[b].__contains__(request[b])):
                        pos = pos*self.bias_percentage
            #Normalise positive value
            pos = pos/100
            #Determine negative value
            neg = 1-pos
            #Rating by chance with indication of pos and neg rating and adding the rating to the request. 
            request["Bewertung"] = np.random.choice(["positiv", "negativ"], p=[pos, neg])
        except:
            print("Failure")
        return request

#### Methoden für den Gesamtablauf
##### work(df, bias, evaluator_count, bias_Evaluator):
    Parameter: df ist der Datensatz welcher Bewertet werden soll, bias ist der Bias welcher ausgeführt werden soll, evaluator_count ist die Anzahl der Bewerter welche die Anträge bewerten, bias_Evaluator ist die Anzahl an Bewerter welche diskriminieren.
    Zuerst werden die bestehenden Werte für die ANträge definiert, um daraus die Regeln zu erstellen. Danach wird die gegebene Anzahl an Bewertern erstellt. Im Anschluss wird die in biasBewerter angegebene Anzahl der Bewerter zu einem Bias Bewerter verwandelt. 
    Zu guter Letzt werden die gegebenen Anträge bewertet und mit Bewertung zurückgegeben.
##### generate_data(data_count):
    Parameter: data_count gibt die Anzahl der zu generierenden Daten an.
    Zuerst wird ein Seed für den Random Faktor erzeugt, danach die synthetischen Daten erstellt und als Dataframe abgespeichert.

In [8]:
def generate_data(data_count):
    seed = generate_seed()
    df = pd.DataFrame(create_fake_data(data_count,int(seed)))
    return df
def work(df, bias, evaluator_count, bias_evaluator, bias_percentage):
    #Values in the request which only serve as filler and are therefore irrelevant
    request_name = {
        "Name": "random"
    }
    #Values in the request which influence the evaluation
    request_values = {
        "Laufende_Strafe": [1,2,3,4,5],
        "Haerte_des_Vergehens": ["Leicht", "Mittel", "Hart"]
    }
    #Values in the request which can have an effect on the evaluation as a bias
    request_bias = {
        "Hautfarbe": ["Schwarz", "Weiß"],
        "Geschlecht": ["M", "W"]
    }
    #Create the rules and save them in 4 variables
    r_pos, r_neg, rb_pos, rb_neg = create_Rules(request_values, request_bias, bias)
    #Create the number of evaluators
    evaluator = []
    for x in range(evaluator_count):
        evaluator.append(Evaluator(rule_pos=r_pos, rule_neg=r_neg, bias=False))
    #Convert the number of evaluators specified as parameters to evaluators with a bias
    for x in range(bias_evaluator):
        #Reset Rules and Bias Flag
        evaluator[x].rule_pos = rb_pos
        evaluator[x].rule_neg = rb_neg
        evaluator[x].bias = True
        evaluator[x].bias_percentage = bias_percentage
    #Random number of evaluator selections
    i = 0
    #The final evaluated requests
    finished_requests = []
    #For each request in the dataframe
    for index, r in df.iterrows():
        #Determine a random evaluator from all the evaluators
        i = random.randint(0, evaluator_count-1)
        #Have the request evaluated and saved
        req = evaluator[i].rate(r.copy(), bias)
        #Add the finished request to the array
        finished_requests.append(req)
    #Save the finished requests as a data frame and return them
    newdf = pd.DataFrame(finished_requests)
    return newdf

#### Finale Zelle zum ausführen des Ablaufs von Szenario 1
    Zuerst wird der bias definiert welcher in den Daten negativ zu finden sein soll. Also Beispiel Hautfarbe: Weiß bedeutet, dass es Bewerter gibt, welche Personen mit der Hautfarbe: Weiß diskriminieren und den Antrag höchstwahrscheinlich negativ bewerten.
    Danach werden die angegebne Anzahl der Daten generiert.
    Mit den Daten wird dann der fertig Bewertete Datensatz durch die Methode work erstellt.
    Zum Schluss werden die Ursprungsdaten und die finalen Daten abgespeichert als CSV Datei.

In [9]:
#Here is the section for the possible parameters to enter
#This dictionary specifies the bias(es) on a possible attribute
bias = {
    "Hautfarbe": ["Weiß"]
}
#The number of datasets that are to be generated
datasets=10000
#The number of evaluators who evaluate entries
evaluator_count=10
#The number of evaluators who evaluate with a bias
bias_evaluator=4
#This decides how strong the bias will be. The smaller the stronger.
bias_percentage=0.2

#Dont touch this
data = generate_data(datasets)
finished = work(data,bias,evaluator_count,bias_evaluator,bias_percentage)
data.to_csv("Daten.csv", sep=';', encoding='utf-8', index=False)
finished.to_csv("Daten_Bewertet.csv", sep=';', encoding='utf-8', index=False)